## MIST Example Notebook

#### Getting started

It is highly recommended that you run MIST out of a Docker container with TensorFlow 2.6.0 or later. Once you start this Jupyter notebook in a container, uncomment the cell below to install some necessary dependencies.

In [1]:
# !pip install -U antspyx
# !pip install -U SimpleITK
# !pip install -U tqdm
# !pip install -U psutil
# !pip install -U tensorflow_addons

#### Import the necessary scripts

The python file 'runtime.py' is the main workhorse for this pipeline. It calls the preprocessing pipeline, network architecture, and prediction metrics used for the global training and inference pipeline.

In [2]:
import json
import os

# Don't show all of the tensorflow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '3'

# Select your GPU (optional)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from runtime import RunTime

/.local/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ev9zjlou because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Quadro RTX 8000, compute capability 7.5


#### Create your user inputs file

To initiate the MIST pipeline, you need to provide it a JSON file with some parameters (i.e., path to data, output paths, and basic training parameters). Below is an example input JSON file to run the MIST pipeline on the LiTS dataset.

In [ ]:
user_params_lits = {'raw_data_dir': '/tf/data/lits/raw/train',
                    'processed_data_dir': '/tf/data/lits/processed/tfrecord',
                    'base_model_name': 'mist_lits_example',
                    'model_dir': '/tf/data/lits/models/mist_example,
                    'prediction_dir': '/tf/data/lits/predictions/mist_example,
                    'raw_paths_csv': '/tf/github/MIST/mist_example/lits_paths.csv',
                    'inferred_params': '/tf/github/MIST/mist_example/lits_inferred_params.json',
                    'results_csv': '/tf/github/MIST/mist_example/lits_results.csv',
                    'modality': 'ct',
                    'mask': ['segmentation_binary'], 
                    'images': {'volume': ['volume']}, 
                    'labels': [0, 1],
                    'final_classes': {'Liver': [1]},
                    'loss': 'gdl', 
                    'model': 'unet', 
                    'pocket': True}

with open(user_params_json_file, 'w') as outfile: 
    json.dump(user_params_lits, outfile)

#### Run MIST pipeline

Once you have your input JSON file, simply run command in the following cell to initiate the MIST training and inference pipeline. Enjoy!

In [ ]:
# Create runtime instance
train = RunTime(json_file)

# Run the runtime instance
train.run(run_preprocess = True)

#### Quick note about the ```run_preprocess``` parameter

The preprocessing pipeline can take quite a while to run depending on the training data. When running the MIST pipeline for the first time, set ```run_preprocess``` to ```True```. However, if you want to try a different model architecture after your initial run, then set ```run_preprocess``` to ```False```. This will skip the preprocessing pipeline and reuse the preprocessed data from the initial run.

In [1]:
# import pandas as pd
# from tqdm import trange
# import subprocess

# lits_df = pd.read_csv('/rsrch1/ip/aecelaya/data/lits/raw/trainingdata.csv')
# for i in trange(len(lits_df)):
#     dest_dir = os.path.abspath('/rsrch1/ip/aecelaya/data/lits/raw/train/lits_{}'.format(i))
#     os.mkdir(dest_dir)
    
#     vol_path = os.path.abspath(lits_df.iloc[i]['ImagePath'])
#     vol_name = 'volume-{}.nii'.format(i)
#     seg_path = os.path.abspath(lits_df.iloc[i]['SegmentationPath'])
#     seg_name = 'segmentation-{}.nii'.format(i)
    
#     cmd1 = 'cp {} {}'.format(vol_path, dest_dir)
#     cmd2 = 'gzip {}'.format(os.path.join(dest_dir, vol_name))
    
#     cmd3 = 'cp {} {}'.format(seg_path, dest_dir)
#     cmd4 = 'gzip {}'.format(os.path.join(dest_dir, seg_name))
    
#     subprocess.call(cmd1, shell = True)
#     subprocess.call(cmd2, shell = True)
#     subprocess.call(cmd3, shell = True)
#     subprocess.call(cmd4, shell = True)

In [4]:
models = ['unet', 'densenet', 'multiresnet', 'hrnet']
pockets = [True, False]
cnt = 1

for model in models:
    for pocket in pockets:
                
        if pocket:
            base_model_name = 'lits_pocket_{}'.format(model)
            model_dir = '/tf/data/lits/models/pocketnet/{}_pocket'.format(model)
            prediction_dir = '/tf/data/lits/predictions/pocketnet/{}_pocket'.format(model)
            results_csv = '/tf/github/MIST/mist/results/lits_pocket_{}_results.csv'.format(model)
            user_params_json_file = os.path.join('/tf/github/MIST/mist/user_params', 
                                                 'lits_pocket_{}_user_params.json'.format(model))
            
        else:
            base_model_name = 'lits_full_{}'.format(model)
            model_dir = '/tf/data/lits/models/pocketnet/{}_full'.format(model)
            prediction_dir = '/tf/data/lits/predictions/pocketnet/{}_full'.format(model)
            results_csv = '/tf/github/MIST/mist/results/lits_full_{}_results.csv'.format(model)
            user_params_json_file = os.path.join('/tf/github/MIST/mist/user_params', 
                                                 'lits_full_{}_user_params.json'.format(model))
        
        user_params_lits = {'raw_data_dir': '/tf/data/lits/raw/train',
                            'processed_data_dir': '/tf/data/lits/processed/tfrecord',
                            'base_model_name': base_model_name,
                            'model_dir': model_dir,
                            'prediction_dir': prediction_dir,
                            'raw_paths_csv': '/tf/github/MIST/mist/path_csv/lits_paths.csv',
                            'inferred_params': '/tf/github/MIST/mist/inferred_params/lits_inferred_params.json',
                            'results_csv': results_csv,
                            'modality': 'ct',
                            'mask': ['segmentation_binary'], 
                            'images': {'volume': ['volume']}, 
                            'labels': [0, 1],
                            'final_classes': {'Liver': [1]},
                            'loss': 'gdl', 
                            'model': model, 
                            'pocket': pocket}

        
        with open(user_params_json_file, 'w') as outfile: 
            json.dump(user_params_lits, outfile)
            
        train = RunTime(user_params_json_file)
        if cnt == 1:
            train.run(run_preprocess = False)
        else:
            train.run(run_preprocess = False)

In [5]:
train = RunTime(json_file)
train.run(run_preprocess = False)

Starting split 1/5
Epoch 1/100
250/250 [==============================] - 248s 931ms/step - loss: 0.2857
INFO:tensorflow:Assets written to: /tf/data/lits/models/pocketnet/unet_pocket/lits_pocket_unet_current_model_split_1/assets
Val loss IMPROVED from inf to 0.2968999445438385
INFO:tensorflow:Assets written to: /tf/data/lits/models/pocketnet/unet_pocket/lits_pocket_unet_best_model_split_1/assets
Epoch 2/100
250/250 [==============================] - 242s 938ms/step - loss: 0.1910
INFO:tensorflow:Assets written to: /tf/data/lits/models/pocketnet/unet_pocket/lits_pocket_unet_current_model_split_1/assets
Val loss of DID NOT improve from 0.2968999445438385
Epoch 3/100


KeyboardInterrupt: 

In [7]:
import numpy as np
t = np.load('test.npy')
import matplotlib.pyplot as plt

plt.imshow(t[0, :, :, 32, :].reshape(64, 64))

In [2]:
import ants
import numpy as np
import SimpleITK as sitk

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-gxsbt_xk because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
x = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/tc.nii.gz')

ki67 = sitk.ReadImage('/tf/remove_later/ki67-docker-cases/10-18-2021/CD.nii.gz')
ki67 = sitk.GetArrayFromImage(ki67).T
ki67 = x.new_image_like(data = ki67)
overlay = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/brainmask.nii.gz')
rc = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/rcmask.nii.gz')

ki67 = ki67.numpy()
overlay = overlay.numpy()
rc = rc.numpy()
overlay -= rc
overlay *= ki67
overlay = x.new_image_like(data = overlay)

ants.plot(x, 
          overlay = overlay, 
          crop = True, 
          nslices = 30, 
          ncol = 6, 
          overlay_alpha = 1.0, 
          cbar = True, 
          title = 'CD Map', 
          filename = '/tf/remove_later/ki67-docker-cases/10-18-2021/cd.png', 
          dpi = 500, 
          overlay_cmap = 'seismic', 
          black_bg = True, 
          axis = 2)

In [70]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics, mixed_precision
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K

################### Loss functions for custom objects ###################
def dice_loss_weighted(y_true, y_pred):
    smooth = 0.0000001

    # (batch size, depth, height, width, channels)
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_true.shape) - 1))
    
    y_true = y_true[..., 0:2]

    Wk = K.sum(y_true, axis = axes)
    Wk = 1. / (K.square(Wk) + 1)

    num = K.sum(K.square(y_true - y_pred), axis = axes)
    den = K.sum(K.square(y_true), axis = axes) + K.sum(K.square(y_pred), axis = axes) + smooth

    return K.sum(Wk * num, axis = -1) / K.sum(Wk * den, axis = -1)

def surface_loss(y_true, y_pred):    
    num_classes = 2
    dtm = y_true[..., num_classes:(2 * num_classes)]
    return K.mean(dtm * y_pred)

def dice_norm_surf_loss_wrapper(alpha):
    def dice_norm_surf_loss(y_true, y_pred):
        return dice_loss_weighted(y_true, y_pred) + (1.0 - alpha) * surface_loss(y_true, y_pred)
    return dice_norm_surf_loss

custom_objects = {'dice_loss_weighted': dice_loss_weighted, 
                  'dice_norm_surf_loss': dice_norm_surf_loss_wrapper(tf.Variable(1.0))}

In [12]:
base_dir = os.path.abspath('/tf/github/NecrosisRecurrence/ki67_docker/tag-hdbet/rcmodels/')
models = os.listdir(base_dir)
for model in models:
    path = os.path.join(base_dir, model)
    new_model_path = os.path.join(base_dir, '{}.h5'.format(model))
    model = load_model(path, custom_objects = custom_objects)
    tf.keras.models.save_model(model, new_model_path)

In [14]:
model = load_model(os.path.join(base_dir, 'rc_model_best_model_split_3.h5'), custom_objects = custom_objects)

In [15]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 12 0                                            
__________________________________________________________________________________________________
conv3d_30 (Conv3D)              (None, 128, 128, 128 1744        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 128, 128, 128 64          conv3d_30[0][0]                  
__________________________________________________________________________________________________
activation_28 (Activation)      (None, 128, 128, 128 0           batch_normalization_28[0][0]     
____________________________________________________________________________________________

In [19]:
weights, _ = model.layers[1].get_weights()
weights.shape

(3, 3, 3, 4, 16)